# JuliaLang: The Ingredients for a Composable Programming Language

Lyndon White   
Research Software Engineer  
Invenia Labs

I am going to tell you some things that may sound counter-intuitive.
I am going suggest, that some of the things Julia that make julia code so reusable,
are because the language has the right combination of missing features, and good features to succeed.

Not just missing features like:
 - Don't allow `eval` into local scope -- it destroys the ability to run an optimizer.
 
But missing features like:
 - Never got round to making it easy to use local modules, outside of packages
 - A type system that can't be used to check correctness

But that these are countered by, or allow for other features:
 - Very easy to create packages
 - Duck-typing, and multiple dispatch, together.

Some questions:

 - How often is code written by grad students reusable by others?
 - How often can you take someone elses types and uses them with your functions and your types? How many lines of code does that take?



## Its easier to create a package than a local module.

Many languages have one module per file,
and you can load that module e.g. via `import Filename` from your current directory.

You can make this work in Julia also, but it is surprisingly fiddly.

What is easy however, is to create and use a package.


### What does making a local module generally give you?

 - Namespacing
 - The feeling you are doing good software engineering
 - Easier to transition later to a package



### What does making a Julia package give you?
 - All the above plus
 - Standard directory structure, `src`, `test` etc
 - Managed dependencies, both what they are, and what versions
 - Easy re-distributivity -- harder to have local state
 - Test-able using package manager's `pkg> test MyPackage` 

And if you use the recommended extension package:
 - Continuous Integration(/s) Setup
 - Code coverage
 - Documentation setup
 - License set

Testing Julia code is important.  
Dynamic language: type system says nothing about correctness.

You will note that many of those were to do with testing.
Testing julia code is important.

This gives us other things too:

 - Community is starting to build bots that do things like testing packages that depend on yours, and make PR to udate compat bounds before release.
 - Before any release of the language itself, all the tests of all registered packages are run to make sure that the language hasn't broken any packages.

### Julia types only give expressivity

 - Julia types don't prove correctness.
 - There are no statements made up front about what methods are valid for a given type.
 - Full static type-checking is impossible
 - but you get duck-typing


Consider on might have a type from the **Ducks** library.

In [3]:
struct Duck end

walk(self) = println("🚶 Waddle")
talk(self) = println("🦆 Quack")

raise_young(self, child) = println("🐤 ➡️ 💧 Lead to water")

raise_young (generic function with 1 method)

and I have some code I want to run, that I wrote:

In [4]:
function simulate_farm(adult_animals, baby_animals)
    for animal in adult_animals
        walk(animal)
        talk(animal)
    end

    parent = first(adult_animals)
    for child in baby_animals
        raise_young(parent, child)
    end
end

simulate_farm (generic function with 1 method)

In [5]:
simulate_farm([Duck(), Duck(), Duck()], [Duck(), Duck()])

🚶 Waddle
🦆 Quack
🚶 Waddle
🦆 Quack
🚶 Waddle
🦆 Quack
🐤 ➡️ 💧 Lead to water
🐤 ➡️ 💧 Lead to water


Ok now I want to extend it with my own type. A Swan

In [7]:
struct Swan end

In [8]:
# Lets test with just 1 first:
simulate_farm([Swan()], [])

🚶 Waddle
🦆 Quack


The **Waddle** was right, but Swans don't **Quack**.

We did some duck-typing -- Swans walk like ducks,
but they don't talk like ducks.


We can solve that with **single dispatch**.

In [12]:
talk(self::Swan) = println("🦢 Hiss")

talk (generic function with 2 methods)

In [13]:
# Lets test with just 1 first:
simulate_farm([Swan()], [])

🚶 Waddle
🦢 Hiss


In [14]:
# Now the whole farm
simulate_farm([Swan(), Swan(), Swan()], [Swan(), Swan()])

🚶 Waddle
🦢 Hiss
🚶 Waddle
🦢 Hiss
🚶 Waddle
🦢 Hiss
🐤 ➡️ 💧 Lead to water
🐤 ➡️ 💧 Lead to water


That's not right. Swans do not lead their young to water.

They carry them

![](https://p1.pxfuel.com/preview/695/84/40/nature-bird-swan-young-royalty-free-thumbnail.jpg)

In [16]:
# Same thing again:
raise_young(self::Swan, child::Swan) = println("🐤 ↗️ 🦢 Carry on back")

raise_young (generic function with 2 methods)

In [17]:
# Now the whole farm
simulate_farm([Swan(), Swan(), Swan()], [Swan(), Swan()])

🚶 Waddle
🦢 Hiss
🚶 Waddle
🦢 Hiss
🚶 Waddle
🦢 Hiss
🐤 ↗️ 🦢 Carry on back
🐤 ↗️ 🦢 Carry on back


Now I want a Farm with mixed poultry.
 - 2 Ducks, a Swan, and 2 baby swans

In [20]:
simulate_farm([Duck(), Duck(), Swan()], [Swan(), Swan()])

🚶 Waddle
🦆 Quack
🚶 Waddle
🦆 Quack
🚶 Waddle
🦢 Hiss
🐤 ➡️ 💧 Lead to water
🐤 ➡️ 💧 Lead to water


Thats not right again. 

What happened?

We had a Duck, raising a baby Swan, and it lead it to water.  

Ducks given baby Swans to raise, will just abandon them.



But how will we code this?

## Option 1: Rewrite the Duck

```julia
function raise_young(self::Duck, child::Any)
    if child isa Swan
        println("🐤😢 Abandon")
    else
        println("🐤 ➡️ 💧 Lead to water")
    end
end
```

### Rewriting the Duck has problems
 - Have to edit someone elses library, to add support for *my* type.
 - This could mean adding a lot of code for them to maintain
 - Does not scale, what if other people wanted to add Chickens, Geese etc.
 
#### Varient: Monkey-patch
 - If the language supports monkey patching, could do it that way
 - but it means copying their code into my library, will run in to issues like not being able to update.
 - Scaled to other people adding new types even worse, since no longer a central canonical source to copy
 
#### Varient: could fork their code
 - That is giving up on code reuse.

## Option 2: Inherit from the Duck

(NB: this example is not valid julia code)
```julia
struct DuckWithSwanSupport <: Duck end

function raise_young(self::DuckWithSwanSupport, child::Any)
    if child isa Swan
        println("🐤😢 Abandon")
    else
        raise_young(upcast(Duck, self), child)
    end
end
```

### Inheriting from the Duck has problems:
 - Have to replace every `Duck` in my code-base with `DuckWithSwanSupport`
 - If I am using other libraries that might return a `Duck` I have to deal with that also

#### Still does not scale.
If someone else implements `DuckWithChickenSupport`, and I want to use both there code and mine, what do?
 - Inherit from both? `DuckWithChickenAndSwanSupport`
 - This is the classic multiple inheritance Diamond problem.
 - It's hard. Even in languages supporting multiple inheritance, they may not support it in a useful way for this without me writing special cases for many things.

### Option 3: Multiple Dispatch

This is clean and easy:

In [26]:
raise_young(parent::Duck, child::Swan) = println("🐤😢 Abandon")

raise_young (generic function with 3 methods)

In [27]:
simulate_farm([Duck(), Duck(), Swan()], [Swan(), Swan()])

🚶 Waddle
🦆 Quack
🚶 Waddle
🦆 Quack
🚶 Waddle
🦢 Hiss
🐤😢 Abandon
🐤😢 Abandon


## But does this happen in the wild?

Turns out it does.

The need to extend operations to act on new combinations of types shows up all the time in scientific computing.  
I suspect it shows up more generally, but we've learned to ignore it.  



If you look at a list of BLAS, methods you will see just this,
but encoded not in the types of the input but in the function name.
E.g.
 - `SGEMM` - matrix matrix multiply
 - `SSYMM` - symmetric matrix matrix multiply
 - ...
 - `ZHBMV` - double complex - hermitian banded matrix vector multiply 

And turns out people keep wanting to make more and more matrix types.
 - Banded Matrixes
 - Block Matrixes
 - Block Banded Matrixes
 - Block Banded Block Banded Matrixs
    - where all the blocks are themselves block banded.


And that is before other things you might like to to to a Matri, which you'd like to encode in its type:
 - Running on a GPU
 - Tracking Operations for AutoDiff
 - Naming dimensions, for easy lookup
 - Distributing over a cluster



These are all important and show up in crucial applications.  
When you start applying things across disciplines, they show up even more.  
Like advancements in Neural Differential Equations, which needs:

 - all the types Machine learning research has invented,
 - and all the types Differential equation solving research has invented,

and wants to use them together.

So its not a reasonable thing for a numerical language to say that they've enumerated all the matrix types you might ever need.

## Inserting a human into the JIT

### Basic functionality of an Tracing JIT:

 - Detect important cases via tracing
 - Compile specialized methods for them

This is called specialization.

### Basic functionalitionality of Julia's JIT:
 - Specialize all methods on all types that they are called on as they are called
 
This is pretty good: its a reasonable assumption that the types are going to an important case.

### What does multiple dispatch add?

It lets a human tell it how that specialization should be done.  
Which can add a lot of information.

### Consider Matrix multiplication.

We have
 - `*(::Dense, ::Dense)`: multiply rows by columns and sum.
    - Takes $O(n^3)$ time
 - `*(::Dense, ::Diagonal)`/`*(::Diagonal, ::Dense)`: column-wise/row-wise scaling. 
    - $O(n^2)$ time.
 - `*(::OneHot, ::Dense)` / `*(::Dense, ::OneHot)`: column-wise/row-wise slicing. 
    - $O(n)$ time.
 - `*(::Identity, ::Dense)` / `*(::Dense, ::Identity)`: no change. 
    - $O(1)$ time.

Converting things to `Dense` in this case gives you the write answer, but much slower.
Converting `Dense` to structured sparse, just gives you the wrong answer.

This is why BLAS etc wanted that information.


## Fast array processing doesn't matter
Anyone can have basic fast array processing by throwing the problem to BLAS, or a GPU.

### Having Array types that are parametric on their scalar types; with ability to be equally fast in both. Matters.

Without this, your array code, and your scalar code can not be disentangled.

BLAS for example does not have this.  
It has a method for each combination of scalar and matrix type.

With this seperation, one can add new scalar types:
 - Dual numbers
 - Measument Error tracking numbers
 - Symbolic Algebra numbers

Without ever having to touch array code, except as a late-stage optimization.

Otherwise, one needs to implement array support into one's scalars, to have reasnable performance at all.

People need to invent new languages.
Its a good time to be inventing new languages.
It's good for the world.


I’ld just really like those new languages to please have:
 - multiple dispatch
 - array types that are parametric on their scalar types, at the type level
 - open classes, so you can add methods to things.